In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # 这一行注释掉就是使用gpu，不注释就是使用cpu
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from pyDOE import lhs
from GaussJacobiQuadRule_V3 import Jacobi, DJacobi, GaussLobattoJacobiWeights, GaussJacobiWeights
import time

In [7]:
np.random.seed(1234)
tf.set_random_seed(1234)

In [8]:
class VPINN:
    def __init__(self, X_u_train, u_train, X_quad, W_quad,\
                 F_exact_total, grid, var_form, layers, activation):
        self.x       = X_u_train
        self.u       = u_train
        self.xquad   = X_quad    # 不需要训练点(xf, f) 只需要求积点和边界点
        self.wquad   = W_quad
        self.F_ext_total = F_exact_total
        self.grid    = grid
        self.var_form= var_form
        self.activation = activation
        self.Nelement = np.shape(self.F_ext_total)[0]    # NE
        self.total_record = []

        self.x_tf   = tf.placeholder(tf.float64, shape=[None, self.x.shape[1]])
        self.u_tf   = tf.placeholder(tf.float64, shape=[None, self.u.shape[1]])

        self.weights, self.biases = self.initialize_NN(layers)
        self.u_NN_pred   = self.net_u(self.x_tf)   # 边界点预测值

        self.lossb = tf.reduce_mean(tf.square(self.u_tf - self.u_NN_pred))
        self.lossv = self.variational_loss()
        self.loss  = lossb_weight * self.lossb + self.lossv

        self.LR = LR
        self.optimizer_Adam = tf.train.AdamOptimizer(self.LR)
        self.train_op_Adam = self.optimizer_Adam.minimize(self.loss)

        self.sess = tf.Session()
        self.init = tf.global_variables_initializer()
        self.sess.run(self.init)

    def initialize_NN(self, layers):
        weights = []
        biases = []
        num_layers = len(layers)
        for l in range(0,num_layers-1):
            W = self.xavier_init(size=[layers[l], layers[l+1]])
            b = tf.Variable(tf.zeros([1,layers[l+1]], dtype=tf.float64), dtype=tf.float64)
            weights.append(W)
            biases.append(b)
        return weights, biases

    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]
        xavier_stddev = np.sqrt(2/(in_dim + out_dim), dtype=np.float64)
        return tf.Variable(tf.truncated_normal([in_dim, out_dim], stddev=xavier_stddev,dtype=tf.float64), dtype=tf.float64)

    def neural_net(self, X, weights, biases):
        num_layers = len(weights) + 1
        H = X
        for l in range(0, num_layers-2):
            W = weights[l]
            b = biases[l]
            H = self.activation(tf.add(tf.matmul(H, W), b))
        W = weights[-1]
        b = biases[-1]
        Y = tf.add(tf.matmul(H, W), b)
        return Y

    def net_u(self, x):
        # tf.concat([tensor1, tensor2, ..., tensorn], axis)
        u = self.neural_net(tf.concat([x], 1), self.weights, self.biases)
        # u = self.neural_net(x, self.weights, self.biases)
        return u

    def net_du(self, x):
        u   = self.net_u(x)
        d1u = tf.gradients(u, x)[0]
        d2u = tf.gradients(d1u, x)[0]
        return d1u, d2u

    # 构造测试函数集
    def Test_fcn(self, N_test, x):
        test_total = []
        for n in range(1, N_test+1):
            test = Jacobi(n+1, 0, 0, x) - Jacobi(n-1, 0, 0, x)
            test_total.append(test)
        return np.asarray(test_total)

    # 返回测试函数的一阶和二阶微分
    def dTest_fcn(self, N_test, x):
        d1test_total = []
        d2test_total = []
        for n in range(1, N_test+1):
            if n==1:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)
            elif n==2:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)
            else:
                d1test = ((n+2)/2)*Jacobi(n,1,1,x) - ((n)/2)*Jacobi(n-2,1,1,x)
                d2test = ((n+2)*(n+3)/(2*2))*Jacobi(n-1,2,2,x) - ((n)*(n+1)/(2*2))*Jacobi(n-3,2,2,x)
                d1test_total.append(d1test)
                d2test_total.append(d2test)
        return np.asarray(d1test_total), np.asarray(d2test_total)

    # variational loss
    def variational_loss(self):
        varloss_total = 0
        for e in range(self.Nelement):
            F_ext_element  = self.F_ext_total[e]          # (60, 1) 一个区域内的Fk值
            Ntest_element  = np.shape(F_ext_element)[0]   # N_testfcn = 60

            x_quad_element = tf.constant(self.grid[e] + (self.grid[e+1] - self.grid[e]) / 2*(self.xquad+1))
            jacobian       = (self.grid[e+1] - self.grid[e]) / 2
            # 测试函数及其微分
            test_quad_element = self.Test_fcn(Ntest_element, self.xquad)
            d1test_quad_element, d2test_quad_element = self.dTest_fcn(Ntest_element, self.xquad)
            # PDE及其微分
            d1u_NN_quad_element, d2u_NN_quad_element = self.net_du(x_quad_element)

            if self.var_form == 1:
                U_NN_element = tf.reshape(tf.stack([-jacobian*tf.reduce_sum(self.wquad*d2u_NN_quad_element*test_quad_element[i]) \
                                                   for i in range(Ntest_element)]), (-1, 1))
            if self.var_form == 2:
                U_NN_element = tf.reshape(tf.stack([ tf.reduce_sum(self.wquad*d1u_NN_quad_element*d1test_quad_element[i]) \
                                                    for i in range(Ntest_element)]), (-1, 1))

            Res_NN_element = U_NN_element - F_ext_element
            loss_element   = tf.reduce_mean(tf.square(Res_NN_element))
            varloss_total += loss_element
        return varloss_total

    def predict(self, x):
        u_pred  = self.sess.run(self.u_NN_pred, {self.x_tf: x})
        return u_pred

    def train(self, nIter, tresh):
        tf_dict = {self.x_tf: self.x, self.u_tf: self.u}
        start_time = time.time()
        for it in range(nIter):
            self.sess.run(self.train_op_Adam, tf_dict)
            loss_value  = self.sess.run(self.loss, tf_dict)
            loss_valueb = self.sess.run(self.lossb, tf_dict)
            loss_valuev = self.sess.run(self.lossv, tf_dict)
            self.total_record.append(np.array([it, loss_value]))
            if loss_value < tresh:
                print('It: %d, Loss: %.3e' % (it, loss_value))
                break
            if it % 100 == 0:
                elapsed = time.time() - start_time
                str_print = 'It: %d, Lossb: %.3e, Lossv: %.3e, Time: %.2f'
                print(str_print % (it, loss_valueb, loss_valuev, elapsed))
        end_time = time.time()
        print("training time %f, loss %f"%(end_time - start_time, loss_value))

In [9]:
#++++++++++++++++++++++++++++
LR = 0.001
Opt_Niter = 125000 + 1
Opt_tresh = 2e-32
var_form  = 2
N_Element = 4
Net_layer = [1] + [40] * 5 + [1]
activation = tf.sin
N_testfcn = 60  # 测试函数个数
N_Quad = 80     # 求积点个数
lossb_weight = 1
#++++++++++++++++++++++++++++
# 测试函数迭代 高斯雅各比迭代 测试函数彼此正交 最终总共构造N_testfcn个测试函数
def Test_fcn(n, x):
   test  = Jacobi(n+1, 0, 0, x) - Jacobi(n-1, 0, 0, x)
   return test
#++++++++++++++++++++++++++++
omega = 8*np.pi
amp = 1
r1 = 80
def u_ext(x):
    utemp = 0.1*np.sin(omega*x) + np.tanh(r1*x)
    return amp*utemp

def f_ext(x):
    gtemp =  -0.1*(omega**2)*np.sin(omega*x) - (2*r1**2)*(np.tanh(r1*x))/((np.cosh(r1*x))**2)
    return -amp*gtemp
#++++++++++++++++++++++++++++
# [求积点.shape=(80,), 权函数.shape=(80,)]
[x_quad, w_quad] = GaussLobattoJacobiWeights(N_Quad, 0, 0)
# 测试函数集 shape=(60, 80)
testfcn = np.asarray([Test_fcn(n, x_quad)  for n in range(1, N_testfcn+1)])

# 区域划分
NE = N_Element
[x_l, x_r] = [-1, 1]
delta_x = (x_r - x_l) / NE
grid = np.asarray([x_l + i*delta_x for i in range(NE+1)])
N_testfcn_total = np.array((len(grid)-1)*[N_testfcn])

# 先验知识
if N_Element == 3:
    grid = np.array([-1, -0.1, 0.1, 1])
    NE = len(grid)-1
    N_testfcn_total = np.array([N_testfcn,N_testfcn,N_testfcn])

# 为何是这个算法？计算Fk
F_ext_total = []
for e in range(NE):
    x_quad_element  = grid[e] + (grid[e+1] - grid[e]) / 2 * (x_quad + 1)
    jacobian = (grid[e+1] - grid[e]) / 2
    N_testfcn_temp  = N_testfcn_total[e]
    testfcn_element = np.asarray([Test_fcn(n, x_quad) for n in range(1, N_testfcn_temp+1)])

    f_quad_element = f_ext(x_quad_element)
    F_ext_element  = jacobian*np.asarray([sum(w_quad*f_quad_element*testfcn_element[i]) for i in range(N_testfcn_temp)])
    F_ext_element  = F_ext_element[:, None] # shape=(60, 1)
    F_ext_total.append(F_ext_element)

F_ext_total = np.asarray(F_ext_total)

#++++++++++++++++++++++++++++
# Training points
X_u_train = np.asarray([-1.0, 1.0])[:, None]
u_train   = u_ext(X_u_train)
#++++++++++++++++++++++++++++
# Quadrature points
X_quad_train = x_quad[:, None]
W_quad_train = w_quad[:, None]
#++++++++++++++++++++++++++++
# Test point
delta_test = 0.001
xtest      = np.arange(-1, 1 + delta_test, delta_test) # (2001,)
data_temp  = np.asarray([[xtest[i], u_ext(xtest[i])] for i in range(len(xtest))]) # (2001, 2)
X_test = data_temp[:, 0][:, None]
u_test = data_temp[:, 1][:, None]

In [10]:
model = VPINN(X_u_train, u_train, X_quad_train, W_quad_train,\
              F_ext_total, grid, var_form, Net_layer, activation)
model.train(Opt_Niter, Opt_tresh)
total_record = model.total_record
u_pred = model.predict(X_test)

It: 0, Lossb: 8.255e-01, Lossv: 1.422e+02, Time: 5.02
It: 100, Lossb: 1.269e-03, Lossv: 1.004e+02, Time: 6.14
It: 200, Lossb: 4.386e-03, Lossv: 2.013e+00, Time: 6.89
It: 300, Lossb: 1.435e-04, Lossv: 4.975e-01, Time: 7.55
It: 400, Lossb: 8.399e-06, Lossv: 2.876e-01, Time: 8.64
It: 500, Lossb: 2.463e-05, Lossv: 9.523e-02, Time: 9.38
It: 600, Lossb: 1.574e-05, Lossv: 5.997e-02, Time: 10.08
It: 700, Lossb: 8.554e-06, Lossv: 4.246e-02, Time: 10.68
It: 800, Lossb: 4.644e-06, Lossv: 3.127e-02, Time: 11.36
It: 900, Lossb: 4.559e-06, Lossv: 2.490e-02, Time: 12.01
It: 1000, Lossb: 1.645e-06, Lossv: 1.810e-02, Time: 12.62
It: 1100, Lossb: 8.933e-07, Lossv: 4.319e-02, Time: 13.29
It: 1200, Lossb: 1.338e-06, Lossv: 1.226e-02, Time: 13.95
It: 1300, Lossb: 6.718e-07, Lossv: 1.100e-02, Time: 14.62
It: 1400, Lossb: 1.772e-07, Lossv: 8.936e-03, Time: 15.29
It: 1500, Lossb: 1.956e-06, Lossv: 7.883e-03, Time: 15.90
It: 1600, Lossb: 1.496e-06, Lossv: 7.289e-03, Time: 16.49
It: 1700, Lossb: 1.117e-05, Loss

KeyboardInterrupt: 

In [ ]:
L2 = np.linalg.norm(u_test - u_pred) / np.linalg.norm(u_test)
print(L2)

In [ ]:
#++++++++++++++++++++++++++++
font = 24

fig, ax = plt.subplots()
plt.tick_params(axis='y', which='both', labelleft='on', labelright='off')
plt.xlabel('$iteration$', fontsize = font)
plt.ylabel('$loss \,\, values$', fontsize = font)
plt.yscale('log')
plt.grid(True)
iteration = [total_record[i][0] for i in range(len(total_record))]
loss_his  = [total_record[i][1] for i in range(len(total_record))]
plt.plot(iteration, loss_his, 'gray')
plt.tick_params( labelsize = 20)
fig.set_size_inches(w=11,h=5.5)
plt.savefig('Results/1d/loss.pdf')
#++++++++++++++++++++++++++++

pnt_skip = 25
fig, ax = plt.subplots()
plt.locator_params(axis='x', nbins=6)
plt.locator_params(axis='y', nbins=8)
plt.xlabel('$x$', fontsize = font)
plt.ylabel('$u$', fontsize = font)
plt.axhline(0, linewidth=0.8, linestyle='-', color='gray')
for xc in grid:
    plt.axvline(x=xc, linewidth=2, ls = '--')
plt.plot(X_test, u_test, linewidth=1, color='r', label=''.join(['$exact$']))
plt.plot(X_test[0::pnt_skip], u_pred[0::pnt_skip], 'k*', label='$VPINN$')
plt.tick_params( labelsize = 20)
legend = plt.legend(shadow=True, loc='upper left', fontsize=18, ncol = 1)
fig.set_size_inches(w=11,h=5.5)
plt.savefig('Results/1d/prediction.pdf')
#++++++++++++++++++++++++++++

fig, ax = plt.subplots()
plt.locator_params(axis='x', nbins=6)
plt.locator_params(axis='y', nbins=8)
plt.xlabel('$x$', fontsize = font)
plt.ylabel('point-wise error', fontsize = font)
plt.yscale('log')
plt.axhline(0, linewidth=0.8, linestyle='-', color='gray')
for xc in grid:
    plt.axvline(x=xc, linewidth=2, ls = '--')
plt.plot(X_test, abs(u_test - u_pred), 'k')
plt.tick_params( labelsize = 20)
fig.set_size_inches(w=11,h=5.5)
plt.savefig('Results/1d/error.pdf')
#++++++++++++++++++++++++++++